# Make PAM tree and clustering

In [1]:
import os
import sys
import glob
import shutil
import numpy as np
import subprocess as sbp
import pandas as pd
from Bio import Seq, SeqIO
import logomaker
import itertools
import matplotlib.pyplot as plt
from collections import Counter
from Bio import SearchIO
import math
from scipy import stats
from matplotlib import cm
from matplotlib import colors
import matplotlib.patches as mpatches
import multiprocessing as mp
from Bio import Phylo
import matplotlib.image as image
from Bio import Phylo
%matplotlib notebook

## Read PAM distance matrix and normalize it

In [2]:
workdir = 'PAM_clustering'

In [34]:
ID = 98
JS_matrix = pd.read_csv(os.path.join(workdir, 'all_PAMs_JS_div_{}_final.tsv'.format(ID)),
                        sep='\t', index_col=0)

In [35]:
JS_matrix_norm = JS_matrix.divide(10) # 10 is the upper bound

In [5]:
matrix_to_write = ['{}\t{}\t{}\n'.format(eval(i)[0],
    eval(i)[1], float(JS_matrix_norm.loc[i])) for i in JS_matrix_norm.index]

In [5]:
pairwise_distance_matrix = os.path.join(workdir, 'pairwise_PAM_JS_matrix_final.tsv')

In [7]:
with open(pairwise_distance_matrix, 'w') as fh:
    fh.write(''.join(matrix_to_write))

## Make tree

In [3]:
cluster_file = os.path.join(workdir, 'JS_PAM_clusters_avg.txt')

In [6]:
command = 'usearch -cluster_aggd {} -treeout {} -clusterout {} -id 0.62 -linkage avg'.format(
    pairwise_distance_matrix, os.path.join(workdir, 'PAM_tree_JS_avg_final.nwk'), cluster_file)
command

'usearch -cluster_aggd PAM_clustering/pairwise_PAM_JS_matrix_final.tsv -treeout PAM_clustering/PAM_tree_JS_avg_final.nwk -clusterout PAM_clustering/JS_PAM_clusters_avg.txt -id 0.62 -linkage avg'

In [10]:
os.system(command)

0

## Make PAM predictions

In [4]:
IDs = [98]

In [5]:
blastdir = 'Spacers_alignment'

In [6]:
all_oriented_flanks = {ID:pd.read_csv(os.path.join(blastdir,
    'realigned_oriented_flanking_regions_{}_clustering_new.tsv'.format(ID)),
    sep='\t', index_col=1) for ID in IDs}

In [7]:
for ID in IDs:
    all_oriented_flanks[ID] = all_oriented_flanks[ID].drop('Unnamed: 0', axis=1)
    all_oriented_flanks[ID].columns = ['Upstream', 'Downstream']

In [8]:
datadir = 'Cas9_tables'

In [9]:
cas9_datasets = {ID:pd.read_csv(os.path.join(datadir,
    'all_new_working_Cas9_clust_{}_oriented.tsv'.format(ID)),
    sep='\t', index_col=0) for ID in IDs}

In [10]:
pam_len = 30
relevant_range = 10

In [11]:
pam_predictions = {ID:{'Upstream':{}, 'Downstream':{}} for ID in IDs}

In [12]:
def make_info_df(seqs, pam_len, pos):
    counters = [Counter() for i in range(pam_len)]
    if pos == 'Downstream':
        for i in range(len(counters)):
            for seq in seqs:
                if i < len(seq):
                    if seq[i]!='N':
                        counters[i][seq[i]] +=1
    else:
        for i in range(1,len(counters)+1):
            for seq in seqs:
                if i <= len(seq):
                    if seq[-i]!='N':
                        counters[-i][seq[-i]] +=1
    freqs = pd.DataFrame(counters).fillna(0)
    freqs = freqs.divide(freqs.sum(axis=1), axis=0).fillna(1.0/freqs.shape[1])
    if not all(freqs.sum(axis=1).apply(lambda x: math.isclose(x, 1.0))):
        raise ValueError
    info = logomaker.transform_matrix(freqs, from_type='probability',
        to_type='information')
    return info

In [13]:
for ID in IDs:
    for i in all_oriented_flanks[ID].index:
        up = all_oriented_flanks[ID].loc[i, 'Upstream']
        down = all_oriented_flanks[ID].loc[i, 'Downstream']
        if pd.notna(up):
            pam_predictions[ID]['Upstream'][i] = make_info_df(up.split(','),
                                                              pam_len, 'Upstream')
        if pd.notna(down):
            pam_predictions[ID]['Downstream'][i] = make_info_df(down.split(','),
                                                                pam_len, 'Downstream')

In [14]:
n_clusters = {ID:len(cas9_datasets[ID]['Cluster ID'].unique()) for ID in IDs}
n_clusters_w_matches = {ID:all_oriented_flanks[ID].shape[0] for ID in IDs}

In [15]:
min_n_flanking_seqs = 10

In [16]:
n_upstream_flanks = {ID:all_oriented_flanks[ID]['Upstream'].apply(
    lambda x: len(x.split(',')) if pd.notna(x) else 0) for ID in IDs}
n_downstream_flanks = {ID:all_oriented_flanks[ID]['Downstream'].apply(
    lambda x: len(x.split(',')) if pd.notna(x) else 0) for ID in IDs}

In [17]:
more_flanks_than_min = {ID:{'Upstream':n_upstream_flanks[ID]>=min_n_flanking_seqs,
    'Downstream':n_downstream_flanks[ID]>=min_n_flanking_seqs} for ID in IDs}

In [18]:
all_info_all_PAMs = {ID:{pos:{i:pam_predictions[ID][pos][i].sum(axis=1).values
    for i in pam_predictions[ID][pos]} for pos in pam_predictions[ID]
    } for ID in IDs}

In [19]:
all_info_all_PAMs = {ID:{pos:{i:all_info_all_PAMs[ID][pos][i] for i in
    all_info_all_PAMs[ID][pos] if more_flanks_than_min[ID][pos].loc[i]} for pos in
    all_info_all_PAMs[ID]} for ID in IDs}

In [20]:
all_ids = {ID:np.unique([k for pos in all_info_all_PAMs[ID] for k in
                     all_info_all_PAMs[ID][pos]]) for ID in IDs}

In [21]:
thresholds = {ID:{k:min(2, max(1, pd.Series(np.concatenate(
    [all_info_all_PAMs[ID][pos][k] for pos in all_info_all_PAMs[ID] if k
     in all_info_all_PAMs[ID][pos]])).quantile(0.75)+1.5*stats.iqr(np.concatenate(
    [all_info_all_PAMs[ID][pos][k] for pos in all_info_all_PAMs[ID] if k
     in all_info_all_PAMs[ID][pos]])))) for k in all_ids[ID]} for ID in IDs}

In [22]:
pam_predicted_positions = {ID:pd.DataFrame({k:{pos:any((all_info_all_PAMs[ID][
    pos][k][0:relevant_range] if pos=='Downstream' else all_info_all_PAMs[ID][
    pos][k][-relevant_range:])>=thresholds[ID][k]) if k in
    all_info_all_PAMs[ID][pos] else False for pos in all_info_all_PAMs[ID]}
    for k in all_ids[ID]}).transpose() for ID in IDs}

In [23]:
has_prediction = {ID:pam_predicted_positions[ID][
    pam_predicted_positions[ID].apply(lambda x: x[0]^x[1], axis=1)] for ID in IDs}

In [24]:
high_confidence_PAM_positions = {ID:has_prediction[ID].apply(
    lambda x: 'Upstream' if x[0] else 'Downstream', axis=1) for ID in IDs}

In [25]:
len(high_confidence_PAM_positions[98])

2546

## Read clusters

In [26]:
clustering = pd.read_csv(cluster_file, sep='\t', header=None,
                         names=['Cluster', 'ID'])

In [27]:
clustering = clustering.groupby('Cluster')['ID'].apply(list)

In [46]:
large_clusters = clustering[clustering.apply(len)>=20]
len(large_clusters)

32

In [29]:
large_clusters.apply(len).sum()

2014

In [30]:
large_clusters.apply(len).sum()/len(high_confidence_PAM_positions[98])

0.7910447761194029

## Make consensus PAMs

In [31]:
all_seqs_reoriented = {}
for i in large_clusters.index:
    all_seqs_reoriented[i] = []
    for Cas9 in large_clusters.loc[i]:
        pos = high_confidence_PAM_positions[ID].loc[Cas9]
        seqs = all_oriented_flanks[ID].loc[Cas9, pos].split(',')
        if pos == 'Upstream':
            seqs = [str(Seq.Seq(s).reverse_complement()) for s in seqs]
        all_seqs_reoriented[i] += seqs

In [32]:
all_info_reoriented = {i:make_info_df(all_seqs_reoriented[i], pam_len, 'Downstream')
                       for i in all_seqs_reoriented}

In [36]:
within_cluster_avg_dist = {}
for i in large_clusters.index:
    pairs = ["('{}', '{}')".format(x,y) for x,y in itertools.combinations(
        np.sort(large_clusters.loc[i]), 2)]
    within_cluster_avg_dist[i] = JS_matrix.loc[pairs].median().iloc[0]

### Sort clusters by size

In [48]:
large_clusters = large_clusters[large_clusters.apply(len).sort_values(
    ascending=False).index]